# Importance Sampling for a European Call

The goal of this exercise is to show how importance sampling can be used to provide a more accurate predictive interval for the price of an option.

The value of a European call option, $ V(t, S_t) $ is defined as $ \mathbb{E}^\mathbb{Q} \left[ e^{-r(T-t)}\max(S_T - K, 0) \right | \mathcal{F}_t] $

That is, the risk neutral expectation of the discounted terminal payoff conditional on all information obtained until the current time.

Consider the following scenario:

$S(0) = 50, \sigma = 0.2, r = 0.06$, and $T = 0.5$

Let us consider a range of strikes $K$ with values 40, 50, 70, and 90.

Intuitively, we expect standard monte carlo to perform well for strikes close to the starting value, 40 and 50 in this case, and worse when the strikes are far from the current option value, in this example 70 and 90. Let's see what happens.

First, as a reference, let's use the black scholes equations to compute the theoretical values as a reference.

In [36]:
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt

import BMS


#Define parameters
T = 0.5
r = 0.06
sigma = 0.2
S0 = 50
Kn = np.array([40,50,70,90])
MC = 10000 #number of monte carlo iterations.
BS_exact = [0 for _ in range(len(Kn))]

for i in range(0, len(Kn)):
    BS_exact[i] = BMS.BMS_price('call', S = S0, K = Kn[i], r = r, q = 0, sigma = sigma, T = T, t = 0)

#Format the output to 4 decimal points
print([ '%.4f' % elem for elem in BS_exact ])

['11.2732', '3.5779', '0.0441', '0.0001']


Next, let's proceed with the standard Monte Carlo simulation

In [47]:

#Allocate space for storage
MCcall = [0 for _ in range(len(Kn))]
CIWidth_MC = [0 for _ in range(len(Kn))]
CI_MCcall = np.zeros(shape = (len(Kn), 2))

#set seed
np.random.seed(seed=233423)

#Sample from the normal distribution
Z = scipy.stats.norm.rvs(loc = 0, scale = 1, size = MC)
# Use standard MC method to estimate the option price
#This is a Geoemtric brownian motion construction
ZForMC = (r - sigma**2 / 2) * T + sigma * np.sqrt(T) * Z;
SFinal = S0 * np.exp(ZForMC);
for i in range(0, len(Kn)):
    K = Kn[i]
    #REMEMBER TO USE numpy.maximum, not np.max. They are two different functions!
    DiscPayoff = np.exp(-r * T) * np.maximum(SFinal - K, 0)
    MCcall[i] = np.mean(DiscPayoff)
    StdDev = np.std(DiscPayoff)
    #Confidence Interval
    CI_MCcall[i,0] = MCcall[i] - 1.96*StdDev/np.sqrt(MC)
    CI_MCcall[i,1] = MCcall[i] + 1.96*StdDev/np.sqrt(MC)
    CIWidth_MC[i] = 1.96 * StdDev / np.sqrt(MC);

print([ '%.4f' % elem for elem in MCcall ])


['11.3164', '3.6071', '0.0556', '0.0000']


Visually, the standard Monte Carlo seems fairly accurate; however, on a percentage basis, we notice that the deviations for the latter 2 strikes seem to be very high.